In [1]:
import os
import time

import tools
import benchmarks as bm
from poseEvaluation import PoseErrorEvaluator
from evo.core import metrics

[DEBUG][2025-06-17 11:11:55,601][log.configure_logging():113]
System info:
Python 3.13.2
Linux-6.2.0-060200-generic-x86_64-with-glibc2.31
i3t-sigma@I3T-Sigma



In [ ]:
benchmark_list = ["XREVA"]
trials = ['data']
# Metrics
delta = 0.1
metric_unit = metrics.Unit.meters

deviceSet = "S1"

max_diff = 0.05
max_null_length = 10

root_dir = ".."
os.chdir(root_dir)
root_dir = "."

In [3]:
poseErrorEvaluator = PoseErrorEvaluator(root_dir, metric_unit=metric_unit, delta=delta, max_diff=0.05, max_null_length=10)

In [4]:
tools.delete_files_in_folder("./logs")
tools.delete_files_in_folder("./data", True)
tools.delete_files_in_folder("./figures")

Skipping: ./logs/distribution (It is not a file)
All files in ./logs deleted successfully.
All files in ./data deleted successfully.
All files in ./figures deleted successfully.


In [ ]:
for i, benchmark in enumerate(benchmark_list):
    benchmarkClass = bm.benchmark_factory[benchmark]
    benchmarkObject = benchmarkClass(root_dir)
    benchmarkObject.generate_script(Set=deviceSet)
    scriptList = benchmarkObject.get_script()

    print("Total {} scripts".format(len(scriptList)))
    print("scriptList: ", scriptList)
    print("="*50)

    
    for j, scriptDict in enumerate(scriptList):    
        time.sleep(0.3)
        benchmark = scriptDict["benchmark"]
        trajectory = scriptDict["trajectory"]
        #benchmarkObject.copy_ground_truth_traj(scriptDict["trajectory"])
        trial = scriptDict["trial"]
            
        # if trial == "set1":
        #     continue

        folder_path = "./logs/distribution"
        if not os.path.exists(folder_path):
            # Create the folder
            os.makedirs(folder_path)
            print("Folder created.")
        else:
            print("Folder already exists.")

        folder_path = scriptDict['script'].split(" ")[-1]
        print("folder_path: ", folder_path)
        if not os.path.exists(folder_path):
            print("Folder not exists. Skipped")
            continue
        
        consoleCommand = "ulimit -s 64000 && " + scriptDict['script']
        print(consoleCommand)
        try:
            tools.runCommand(consoleCommand, consoleOutputEncoding="utf-8", timeout=120)
            #os.system(scriptDict['script'])
        except:
            print("="*50)
            print("{}-{}-{} Timeout. Skipped".format(benchmark, trajectory, trial))
            print("="*50)
            continue
        time.sleep(0.3)
        
        benchmarkObject.copy_ground_truth_traj(trajectory, trial)
        benchmarkObject.process_raw_SLAM_data(benchmark,trajectory,trial)
        time.sleep(0.3)
        
        df, df_traj = benchmarkObject.process_raw_SLAM_data(benchmark,trajectory,trial)
        time.sleep(0.3)
        
        poseErrorEvaluator.load_trajectory(benchmark,trajectory,trial)
        traj_est, traj_ref = poseErrorEvaluator.get_traj_w_gt()
        
        rpe_metric = poseErrorEvaluator.calculate_RE(pose_relation=metrics.PoseRelation.point_distance)
        ape_metric = poseErrorEvaluator.calculate_APE(pose_relation=metrics.PoseRelation.point_distance)
        poseErrorEvaluator.save_error_csv(trajectory=trajectory, trial=trial)
        
        poseErrorEvaluator.merge_feature_with_label(benchmark,trajectory,trial)
        merged_df = poseErrorEvaluator.get_feature_w_label()
        error_df = poseErrorEvaluator.get_error_df()
        
        # poseErrorEvaluator.plot_aligned_trajectory(traj_est, traj_ref, benchmark, trajectory, trial)
        # poseErrorEvaluator.plot_trajectory(traj_est, traj_ref, benchmark, trajectory, trial)
        # poseErrorEvaluator.plot_error(error_df, delta, benchmark, trajectory, trial, plot_error_min = -0.01, plot_error_max=0.1)
        
        poseErrorEvaluator.plot_trajectory_with_error(rpe_metric, traj_ref, traj_est,
                                                    root_dir, trajectory, trial)

        merged_df.to_csv("{}/Datasets/{}/{}/orb_combined.csv".format(root_dir,trajectory,trial), sep=',', index=False, header=True)
        time.sleep(0.3)
        
        print("="*50)
        print("="*50)


Debug: trajs: S2
Total 16 scripts
scriptList:  [{'benchmark': 'XREVA', 'trajectory': 'S2_Side_Featureless_50', 'trial': 'set1', 'script': './Examples/Stereo-Inertial/stereo_inertial_customized ./Vocabulary/ORBvoc.txt ./Examples/Stereo-Inertial/stereo_inertial_customized.yaml ./Datasets/S2_Side_Featureless_50/set1/ ./Datasets/S2_Side_Featureless_50/set1/sensor/timestamp.txt'}, {'benchmark': 'XREVA', 'trajectory': 'S2_Side_Featureless_50', 'trial': 'set2', 'script': './Examples/Stereo-Inertial/stereo_inertial_customized ./Vocabulary/ORBvoc.txt ./Examples/Stereo-Inertial/stereo_inertial_customized.yaml ./Datasets/S2_Side_Featureless_50/set2/ ./Datasets/S2_Side_Featureless_50/set2/sensor/timestamp.txt'}, {'benchmark': 'XREVA', 'trajectory': 'S2_Side_Featureless_50', 'trial': 'set3', 'script': './Examples/Stereo-Inertial/stereo_inertial_customized ./Vocabulary/ORBvoc.txt ./Examples/Stereo-Inertial/stereo_inertial_customized.yaml ./Datasets/S2_Side_Featureless_50/set3/ ./Datasets/S2_Side_Fea

In [6]:
scriptList

[{'benchmark': 'XREVA',
  'trajectory': 'S2_Side_Featureless_50',
  'trial': 'set1',
  'script': './Examples/Stereo-Inertial/stereo_inertial_customized ./Vocabulary/ORBvoc.txt ./Examples/Stereo-Inertial/stereo_inertial_customized.yaml ./Datasets/S2_Side_Featureless_50/set1/ ./Datasets/S2_Side_Featureless_50/set1/sensor/timestamp.txt'},
 {'benchmark': 'XREVA',
  'trajectory': 'S2_Side_Featureless_50',
  'trial': 'set2',
  'script': './Examples/Stereo-Inertial/stereo_inertial_customized ./Vocabulary/ORBvoc.txt ./Examples/Stereo-Inertial/stereo_inertial_customized.yaml ./Datasets/S2_Side_Featureless_50/set2/ ./Datasets/S2_Side_Featureless_50/set2/sensor/timestamp.txt'},
 {'benchmark': 'XREVA',
  'trajectory': 'S2_Side_Featureless_50',
  'trial': 'set3',
  'script': './Examples/Stereo-Inertial/stereo_inertial_customized ./Vocabulary/ORBvoc.txt ./Examples/Stereo-Inertial/stereo_inertial_customized.yaml ./Datasets/S2_Side_Featureless_50/set3/ ./Datasets/S2_Side_Featureless_50/set3/sensor/tim